In [2]:
import shutil
import os
from cgitb import html
import pandas as pd 
import re
import time
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# 控制選單
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def crawl_stock_tw_v1(_ticker, _year):
    url = r'https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html' 
    s = Service(r'C:\SDK\chromedriver_win32\chromedriver.exe')

    browser = webdriver.Chrome(service=s) 
    browser.maximize_window() 
    browser.get(url) 
    sleep(1)

    browser.find_element(by=By.XPATH, value='//*[@id="main-form"]/div/div/form/input').send_keys(str(_ticker))

    Select(browser.find_element(by=By.NAME, value='yy')).select_by_value(_year)

    browser.find_element(by=By.XPATH, value='//*[@id="main-form"]/div/div/form/a[2]').click()

    for i in range(12):
        # 選取表單_設定月分
        Select(browser.find_element(by=By.NAME, value='mm')).select_by_index(i)
        # 點擊查詢按鈕
        browser.find_element(by=By.XPATH, value='//*[@id="main-form"]/div/div/form/a[2]').click()
        # 點擊下載按鈕
        csv = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "csv")))
        time.sleep(5)
        csv.click()

    time.sleep(1)
    browser.close()

In [4]:
def crawl_stock_tw(_ticker, _year):

    s = Service(r'C:\SDK\chromedriver_win32\chromedriver.exe')
    browser = webdriver.Chrome(service=s) 
    browser.maximize_window() 

    for i in range(1, 13):
        url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date={}{:02}01&stockNo={}'.format(_year, i, _ticker)
        browser.get(url) 
        sleep(5)

    browser.close()

In [5]:
stock = 3169
# 判定 os_test 資料夾是否存在
if not os.path.isdir('data_unmerged'): 
    os.mkdir('data_unmerged')
# 取得當前路徑
destination = os.getcwd() + '\\data_unmerged' 

source = os.listdir(r"C:\Users\User\Downloads")
#過濾其他檔案，只保留csv檔
source = [line for line in source if str(stock) in line] 
print(source)

# 移動下載檔案進入資料夾
for s in source:
    path = os.path.join(r"C:\Users\User\Downloads", s)
    shutil.move(path, os.path.join(destination, s))

[]


In [6]:
def move_stock_tw(_ticker):
    # 判定 os_test 資料夾是否存在
    if not os.path.isdir('data_unmerged'): 
        os.mkdir('data_unmerged')
    # 取得當前路徑
    destination = os.getcwd() + '\\data_unmerged' 

    source = os.listdir(r"C:\Users\User\Downloads")
    #過濾其他檔案，只保留csv檔
    source = [line for line in source if str(_ticker) in line] 
    print(source)

    # 移動下載檔案進入資料夾
    for s in source:
        path = os.path.join(r"C:\Users\User\Downloads", s)
        shutil.move(path, os.path.join(destination, s))

In [7]:
stock = 3169
path = os.getcwd() + '\\data_unmerged\\'
source = os.listdir(path)
source = [line for line in source if str(stock) in line]

res = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

for s in source:
    print(s)
    df = pd.read_csv(destination + s, encoding='Big5', skiprows=1)
    df = df.iloc[:-4, :7]
    df.columns = ['Date', 'Volume', 'b', 'Open', 'High', 'Low', 'Close']
    df.drop('b', axis = 1, inplace = True) 
    df = df.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df.loc[:, 'Date'] = df.loc[:, 'Date'].str.split('/').apply(lambda x:(str(int(x[0]) + 1911)) + '/' + x[1].removeprefix('0') + '/' + x[2].removeprefix('0'))
    df.loc[:, 'Volume'] = df.loc[:, 'Volume'].str.replace(',', '').astype(int) // 1000
    res = res.append(df, ignore_index = True)

res['Date'] = pd.to_datetime(res['Date'])
res.sort_values(by='Date', ascending=False)

# 判定 os_test 資料夾是否存在
if not os.path.isdir('data'): 
    os.mkdir('data')

res.to_csv(os.getcwd() + '\\data\\filename.csv', index = False, encoding='utf-8')

In [8]:
def merge_stock_tw(_ticker):
    
    # 判定 data 資料夾是否存在
    if not os.path.isdir('data'): 
        os.mkdir('data')
    
    path = os.getcwd() + '\\data_unmerged\\'
    source = os.listdir(path)
    source = [line for line in source if str(_ticker) in line]
    
    if os.path.isfile(r'./data/{}.csv'.format(str(_ticker))): 
        res = pd.read_csv(os.getcwd() + '\\data\{}.csv'.format(str(_ticker)))
    else:
        res = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    for s in source:
        print(s)
        df = pd.read_csv(destination + s, encoding='Big5', skiprows=1)
        df = df.iloc[:-4, :7]
        df.columns = ['Date', 'Volume', 'b', 'Open', 'High', 'Low', 'Close']
        df.drop('b', axis = 1, inplace = True) 
        df = df.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df.loc[:, 'Date'] = df.loc[:, 'Date'].str.split('/').apply(lambda x:(str(int(x[0]) + 1911)) + '/' + x[1].removeprefix('0') + '/' + x[2].removeprefix('0'))
        df.loc[:, 'Volume'] = df.loc[:, 'Volume'].str.replace(',', '').astype(int) // 1000
        res = res.append(df, ignore_index = True)
    
    
    res['Date'] = pd.to_datetime(res['Date'])
    res = res.drop_duplicates(subset='Date', keep='first')
    res.sort_values(by='Date', ascending=True, inplace=True)
    
    res.to_csv(os.getcwd() + '\\data\\' + str(_ticker) + '.csv', index = False, encoding='utf-8')

In [149]:
stock = input("查詢股票代碼 -> ")
year = input("查詢年份 -> ")

crawl_stock_tw_v1(stock, year)
move_stock_tw(stock)
merge_stock_tw(stock)


查詢股票代碼 -> 2409
查詢年份 -> 2019
['STOCK_DAY_2409_201901.csv', 'STOCK_DAY_2409_201902.csv', 'STOCK_DAY_2409_201903.csv', 'STOCK_DAY_2409_201904.csv', 'STOCK_DAY_2409_201905.csv', 'STOCK_DAY_2409_201906.csv', 'STOCK_DAY_2409_201907.csv', 'STOCK_DAY_2409_201908.csv', 'STOCK_DAY_2409_201909.csv', 'STOCK_DAY_2409_201910.csv', 'STOCK_DAY_2409_201911.csv', 'STOCK_DAY_2409_201912.csv']
           Date   Open   High    Low  Close  Volume
0    2020-01-02  10.15  10.20  10.10  10.15   34118
1    2020-01-03  10.20  10.25   9.89  10.05   68859
2    2020-01-06  10.10  10.20   9.89   9.96   67039
3    2020-01-07  10.05  10.20   9.98  10.05   66052
4    2020-01-08   9.96  10.35   9.85  10.35  145245
..          ...    ...    ...    ...    ...     ...
484  2021-12-24  22.45  22.60  22.25  22.45   49699
485  2021-12-27  22.50  23.00  22.50  22.70   72941
486  2021-12-28  22.90  23.00  22.60  22.65   51370
487  2021-12-29  22.70  22.85  22.50  22.75   39207
488  2021-12-30  22.80  23.15  22.75  22.90   84382